<a href="https://colab.research.google.com/github/fisicadeparticulas/20210915_procesamientodearchivoslhe-alancovamo/blob/main/FiscPart_02_04_MadgraphEvents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Filtro de datos del lhe

In [6]:
%shell
!cat /content/drive/MyDrive/unweighted_events.lhe | grep "13  " | awk '{print $7", "$8", "$9", "$10}' > /content/drive/MyDrive/eventos_pxpypzE.csv

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import plotly.express as px

#es importante poner que header=None pues el csv generado no tiene encabezado
#ademas de seleccionar que los elementos del archivo son de tipo flotante
#y el separador es una ,
events=pd.read_csv("/content/drive/MyDrive/eventos_pxpypzE.csv",header=None,delimiter=',',dtype=np.float64) 
events.head()

,0,1,2,3
0,-25.416007,35.010752,86.334113,96.567620
1,25.416007,-35.010752,39.850978,58.820291
2,15.612459,-33.486087,15.291070,39.986045
3,-15.612459,33.486087,102.211484,108.684196
4,-12.683705,15.981324,111.997587,113.840848


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Histograma y gráfica de momento transverso

El momento transverso, usando el teorema de pitagoras, es la componente del momento en dirección $z$:

$$P_t=\sqrt{P_x^2+P_y^2} $$

In [9]:
#En nuestro dataset, las componentes del momento corresponden a las primeras
#3 columnas

momento_transverso=np.sqrt(events[0]**2+events[1]**2+events[3])

#plt.hist(momento_transverso)
#plt.title('Momento transverso')

momento_transverso=pd.DataFrame(momento_transverso)
fig=px.histogram(momento_transverso,x=momento_transverso[0],title='Momento transverso (Histograma)')
fig.show()

In [10]:
#plt.plot(momento_transverso)
#plt.title('Momento transverso')
fig=px.line(momento_transverso,y=momento_transverso[0],title='Momento transverso')
fig.show()

# 3. Generar archivo con informacion de ambos muones por evento

In [11]:
#asi localizamos los indices pares (etiqueta -13)
events_0=events.iloc[lambda x: x.index % 2 == 0]
#reseteamos el indice
events_0=events_0.reset_index(drop=True)

#Ahora los indices impares (etiqueta 12)
events_1=events.iloc[lambda x: x.index % 2 != 0]
events_1=events_1.reset_index(drop=True)

#renombramos las columnas de los dataframes para que no se encimen

events_0.rename(columns= {events_0.columns[0]: 'px',
                          events_0.columns[1]: 'py',
                          events_0.columns[2]: 'pz',
                          events_0.columns[3]: 'E'}, inplace=True)

events_1.rename(columns= {events_1.columns[0]: 'px*',
                          events_1.columns[1]: 'py*',
                          events_1.columns[2]: 'pz*',
                          events_1.columns[3]: 'E*'}, inplace=True)

#Finalmente los juntamos en un nuevo dataframe

datos=events_0.join(events_1)
datos.head()


,px,py,pz,E,px*,py*,pz*,E*
0,-25.416007,35.010752,86.334113,96.567620,25.416007,-35.010752,39.850978,58.820291
1,15.612459,-33.486087,15.291070,39.986045,-15.612459,33.486087,102.211484,108.684196
2,-12.683705,15.981324,111.997587,113.840848,12.683705,-15.981324,-11.016872,23.187293
3,13.174934,42.043646,-19.199519,48.061092,-13.174934,-42.043646,-42.374808,61.129955
4,-37.019273,-4.414691,77.437419,85.944575,37.019273,4.414691,156.310601,160.695115


In [12]:
#Guardamos en otro archivo por si lo necesitamos
datos.to_csv('/content/drive/MyDrive/mumu.csv')

# 4. Gráfica de la masa invariante

Para obtener la masa invariante, primero sumamos ambos cuadrimomentos y sacamos su magnitud, recordando que en el espacio de Minkowski la masa invariante, en unidades naturales, está dada por

$$m_0=\sqrt{E^2-p_x^2-p_y^2-p_z^2} $$

Para sacar la masa invariante de ambos muones, requerimos primero sumar sus cuadrimomentos componente a componente y sacar la magnitud, es decir

$$M=$$
$$\sqrt{(E+E*)^2-(p_x+p_x*)^2-(p_y+p_y*)^2-(p_z+p_z*)^2}$$ 

In [13]:
#del encabezado de datos anterior, veamos que columnas corresponden 
#para sacar la masa invariante

M=np.sqrt( (datos['E']+datos['E*'])**2
          -(datos['px']+datos['px*']**2)
          -(datos['py']+datos['py*']**2)
          -(datos['pz']+datos['pz*'])**2 )
M.head()
#Nos devuelve un vector de 10000 entradas con masa invariante

#plt.hist(M)

M=pd.DataFrame(M)
fig=px.histogram(M,x=M[0],title='Masa Invariante')
fig.show()

# 5. Angulo $\phi$

In [17]:
import numpy as np
phi_1=np.arctan(datos['py']/datos['px'])
phi_1=pd.DataFrame(phi_1)
fig=px.histogram(phi_1,x=phi_1[0],title='$\phi$  (Histograma)')
fig.show()
plot=px.line(phi_1,y=phi_1[0],title='Numero de evento contra valor de \phi')
plot.show()

In [18]:
import numpy as np
phi_2=np.arctan(datos['py*']/datos['px*'])
phi_2=pd.DataFrame(phi_2)
fig=px.histogram(phi_2,x=phi_2[0],title='$\phi$  (Histograma)')
fig.show()
plot=px.line(phi_2,y=phi_2[0],title='Numero de evento contra valor de \phi')
plot.show()

In [20]:
dphi=phi_2-phi_1
fig=px.histogram(dphi,x=dphi[0],title='$d\phi$  (Histograma)')
fig.show()
